# 3D Glasses Reconstruction Pipeline

This notebook demonstrates the complete workflow for reconstructing 3D glasses models from 2D images using the Hunyuan3D-2 model.

## 1. Setup and Imports

In [ ]:
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
import trimesh
from PIL import Image
from tqdm.notebook import tqdm

# Add the project root to the path
sys.path.append('..')

# Import project modules
from src.data import GlassesDataset, get_transforms
from src.models import HunyuanAdapter, GlassesReconstruction
from src.metrics import chamfer_distance, earth_movers_distance, iou_3d
from src.utils import visualize_mesh, visualize_point_cloud, visualize_comparison, save_mesh

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 2. Load the Dataset

In [ ]:
# Set data directory
data_dir = '../data'

# Create data transforms
transforms = get_transforms(image_size=256, split='test')

# Create dataset
dataset = GlassesDataset(
    data_dir=data_dir,
    split='test',
    transform=transforms['image'],
    target_transform=transforms['model'],
    image_size=256,
    max_samples=10  # Limit to 10 samples for demonstration
)

print(f"Loaded {len(dataset)} samples")

## 3. Visualize Sample Images

In [ ]:
# Visualize a few sample images
fig, axes = plt.subplots(1, 5, figsize=(20, 4))

for i in range(5):
    if i < len(dataset):
        sample = dataset[i]
        image = sample['image']
        
        # Convert tensor to numpy array for visualization
        image_np = image.permute(1, 2, 0).numpy()
        
        # Denormalize image
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        image_np = image_np * std + mean
        image_np = np.clip(image_np, 0, 1)
        
        # Display image
        axes[i].imshow(image_np)
        axes[i].set_title(f"Sample {i+1}")
        axes[i].axis('off')

plt.tight_layout()
plt.show()

## 4. Load the Model

In [ ]:
# Create the model
base_model = HunyuanAdapter(model_path='tencent/Hunyuan3D-2', device=device)
model = GlassesReconstruction(
    base_model=base_model,
    num_classes=10,
    feature_dim=512
)
model = model.to(device)

# Load checkpoint if available
checkpoint_path = '../checkpoints/best_model.pth'
if os.path.exists(checkpoint_path):
    print(f"Loading checkpoint from {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
else:
    print("No checkpoint found, using untrained model")

# Set model to evaluation mode
model.eval()

## 5. Generate 3D Reconstructions

In [ ]:
# Create results directory
results_dir = '../results/notebook'
os.makedirs(results_dir, exist_ok=True)

# Generate 3D reconstructions for a few samples
num_samples = min(5, len(dataset))
reconstructions = []

for i in tqdm(range(num_samples), desc="Generating reconstructions"):
    sample = dataset[i]
    image = sample['image'].unsqueeze(0).to(device)  # Add batch dimension
    
    # Generate mesh
    with torch.no_grad():
        mesh = model.generate_mesh(
            image=image,
            with_texture=True,
            output_path=os.path.join(results_dir, f"sample_{i+1}.obj")
        )
    
    reconstructions.append(mesh)

## 6. Visualize Reconstructions

In [ ]:
# Visualize the reconstructions
for i, mesh in enumerate(reconstructions):
    print(f"Sample {i+1}:")
    visualize_mesh(mesh, figsize=(8, 8))

## 7. Evaluate Reconstructions

In [ ]:
# Evaluate reconstructions against ground truth if available
metrics = {
    'chamfer_distance': [],
    'earth_movers_distance': [],
    'iou_3d': []
}

for i in range(num_samples):
    sample = dataset[i]
    gt_mesh_path = sample['model_path']
    
    if gt_mesh_path is not None and os.path.exists(gt_mesh_path):
        # Load ground truth mesh
        gt_mesh = trimesh.load(gt_mesh_path)
        
        # Get predicted mesh
        pred_mesh = reconstructions[i]
        
        # Compute Chamfer distance
        cd = chamfer_distance(
            pred_mesh.vertices,
            gt_mesh.vertices,
            bidirectional=True,
            reduction='mean'
        )
        metrics['chamfer_distance'].append(float(cd))
        
        # Compute Earth Mover's Distance
        emd = earth_movers_distance(
            pred_mesh.vertices,
            gt_mesh.vertices,
            reduction='mean'
        )
        metrics['earth_movers_distance'].append(float(emd))
        
        # Compute IoU
        iou = iou_3d(pred_mesh, gt_mesh)
        metrics['iou_3d'].append(float(iou))
        
        # Visualize comparison
        print(f"Sample {i+1}:")
        visualize_comparison(
            meshes=[pred_mesh, gt_mesh],
            titles=["Predicted", "Ground Truth"],
            figsize=(15, 5)
        )
        
        # Print metrics
        print(f"Chamfer Distance: {cd:.4f}")
        print(f"Earth Mover's Distance: {emd:.4f}")
        print(f"IoU: {iou:.4f}")
        print("\n")

## 8. Compute Average Metrics

In [ ]:
# Compute average metrics
avg_metrics = {
    'chamfer_distance': np.mean(metrics['chamfer_distance']) if metrics['chamfer_distance'] else float('nan'),
    'earth_movers_distance': np.mean(metrics['earth_movers_distance']) if metrics['earth_movers_distance'] else float('nan'),
    'iou_3d': np.mean(metrics['iou_3d']) if metrics['iou_3d'] else float('nan')
}

print("Average Metrics:")
print(f"Chamfer Distance: {avg_metrics['chamfer_distance']:.4f}")
print(f"Earth Mover's Distance: {avg_metrics['earth_movers_distance']:.4f}")
print(f"IoU: {avg_metrics['iou_3d']:.4f}")

## 9. Custom Image Reconstruction

In [ ]:
# Function to reconstruct a custom image
def reconstruct_custom_image(image_path, output_path=None):
    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')
    transform = get_transforms(image_size=256, split='test')['image']
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    # Generate mesh
    with torch.no_grad():
        mesh = model.generate_mesh(
            image=image_tensor,
            with_texture=True,
            output_path=output_path
        )
    
    # Display original image
    plt.figure(figsize=(5, 5))
    plt.imshow(image)
    plt.title("Input Image")
    plt.axis('off')
    plt.show()
    
    # Visualize mesh
    visualize_mesh(mesh, figsize=(8, 8))
    
    return mesh

# Example usage (uncomment and provide a path to your custom image)
# custom_image_path = "path/to/your/image.jpg"
# custom_output_path = os.path.join(results_dir, "custom_reconstruction.obj")
# custom_mesh = reconstruct_custom_image(custom_image_path, custom_output_path)

## 10. Conclusion

This notebook demonstrated the complete workflow for reconstructing 3D glasses models from 2D images using the Hunyuan3D-2 model. The pipeline includes:

1. Loading and preprocessing the dataset
2. Loading the model
3. Generating 3D reconstructions
4. Visualizing the reconstructions
5. Evaluating the reconstructions using various metrics
6. Reconstructing custom images

The model can be further improved by training on a larger dataset of glasses images and their corresponding 3D models.